<a href="https://colab.research.google.com/github/dushyilluminati/Proj3/blob/main/Project_3_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Dataset - Project 3.csv to Dataset - Project 3.csv


In [ ]:
data=pd.read_csv('Dataset - Project 3.csv')

In [ ]:
print(data.describe())

       Transaction_Amount  Transaction_Time
count          100.000000        100.000000
mean          5096.659400         11.440000
std           2900.836326          7.092919
min             18.200000          0.000000
25%           3035.920000          5.000000
50%           5228.380000         12.000000
75%           7561.997500         17.000000
max           9981.860000         23.000000


In [ ]:
print(data.head())

  Transaction_ID Customer_ID  Transaction_Amount  Transaction_Time Location  \
0        TRANS_1     CUST_45             4305.86                14   Mobile   
1        TRANS_2     CUST_42             9233.85                 0   Mobile   
2        TRANS_3     CUST_33             9960.74                 6   Mobile   
3        TRANS_4     CUST_35             7315.20                 4   Online   
4        TRANS_5      CUST_6             7672.34                23   Mobile   

   Flagged_Fraudulent  
0               False  
1                True  
2               False  
3               False  
4                True  


In [ ]:
data = data.dropna()

In [ ]:
data['Transaction_Hour'] = pd.to_datetime(data['Transaction_Time']).dt.hour
data = pd.get_dummies(data, columns=['Location'], drop_first=True)


In [ ]:
scaler = StandardScaler()
data['Transaction_Amount'] = scaler.fit_transform(data[['Transaction_Amount']])


In [ ]:
X = data.drop(columns=['Transaction_ID', 'Customer_ID', 'Transaction_Time', 'Flagged_Fraudulent'])
y = data['Flagged_Fraudulent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=75, stratify=y)


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [ ]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f'Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%')


[[ 0 22]
 [ 0 28]]
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        22
        True       0.56      1.00      0.72        28

    accuracy                           0.56        50
   macro avg       0.28      0.50      0.36        50
weighted avg       0.31      0.56      0.40        50

Accuracy: 56.00%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

       False       0.50      0.45      0.48        22
        True       0.60      0.64      0.62        28

    accuracy                           0.56        50
   macro avg       0.55      0.55      0.55        50
weighted avg       0.56      0.56      0.56        50



In [ ]:
pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
from collections import Counter
print("Before SMOTE:", Counter(y_train))
print("After SMOTE:", Counter(y_train_smote))

Before SMOTE: Counter({True: 28, False: 22})
After SMOTE: Counter({False: 28, True: 28})


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees
    'max_depth': [5, 10, 15],         # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum samples to split
    'min_samples_leaf': [1, 2, 5]     # Minimum samples in each leaf
}

In [ ]:
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

In [ ]:
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_smote, y_train_smote)

# Best parameters
print("Best Parameters:", grid_search.best_params_)
best_rf = grid_search.best_estimator_

Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
best_rf.fit(X_train_smote, y_train_smote)

# Predictions
y_pred = best_rf.predict(X_test)

# Evaluation
print("Optimized Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Optimized Random Forest Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

       False       0.50      0.55      0.52        22
        True       0.62      0.57      0.59        28

    accuracy                           0.56        50
   macro avg       0.56      0.56      0.56        50
weighted avg       0.56      0.56      0.56        50

